In [3]:
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import VotingClassifier

In [4]:
# Load dataset from directory
train_dir = 'datasets/train'
test_dir = 'datasets/test'

# Load train and test datasets
train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='int', 
    shuffle=True,
    seed=123 
)

test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='int',
    shuffle=False, 
    seed=123
)

Found 1034 files belonging to 3 classes.


I0000 00:00:1727213922.058643    7737 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727213922.225710    7737 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727213922.230445    7737 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727213922.235343    7737 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Found 128 files belonging to 3 classes.


In [5]:
# Load pre-trained CNN model
cnn_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [6]:
# Function to extract features from the images using CNN
def extract_features(dataset, cnn_model):
    features = []
    labels = []

    for images, lbls in dataset:
        try:
            feature_batch = cnn_model.predict(images)
            features.append(feature_batch)
            labels.append(lbls.numpy()) # Convert labels to numpy arrays
        except Exception as e:
            print("Error processing a batch:", e)
            continue

    features = np.vstack(features) # Shape to (samples, features)
    labels = np.concatenate(labels) # Combine all label batches
    return features, labels

In [7]:
# Extract features from train and test sets
train_features, train_labels = extract_features(train_dataset, cnn_model)
test_features, test_labels = extract_features(test_dataset, cnn_model)

I0000 00:00:1727213925.029714   16516 service.cc:146] XLA service 0x7f2d80003af0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727213925.029741   16516 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2024-09-24 22:38:45.048561: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-24 22:38:45.147930: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-09-24 22:38:46.435972: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.04GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-09-24 22:38:47.286554: W external/local_tsl/tsl/framework/bfc_allocator.cc:291]

1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


I0000 00:00:1727213934.789141   16516 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━

2024-09-24 22:39:05.436067: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


2024-09-24 22:39:06.336414: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
# Flatten the features
train_features = train_features.reshape(train_features.shape[0], -1)
test_features = test_features.reshape(test_features.shape[0], -1)

In [9]:
# Standardize the features
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
test_features_scaled = scaler.transform(test_features)

In [10]:
# Define the SVM and Logistic Regression classifiers
svm_model = SVC(kernel='linear', probability=True)
logr_model = LogisticRegression(max_iter=1000)

In [11]:
# Training the models individually
svm_model.fit(train_features_scaled, train_labels)
logr_model.fit(train_features_scaled, train_labels)

LogisticRegression(max_iter=1000)

In [12]:
# Creating a voting classifier with soft voting for average probabilties sum
voting_model = VotingClassifier(estimators=[('svm', svm_model), ('logreg', logr_model)], voting='soft')

In [13]:
# Train the Voting Classifier
voting_model.fit(train_features_scaled, train_labels)

VotingClassifier(estimators=[('svm', SVC(kernel='linear', probability=True)),
                             ('logreg', LogisticRegression(max_iter=1000))],
                 voting='soft')

In [14]:
test_predictions = voting_model.predict(test_features_scaled)
train_predictions = voting_model.predict(train_features_scaled)

In [ ]:
print("Test accuracy: {:.2f}".format(accuracy_score(test_labels, test_predictions)))
print("Training accuracy: {:.2f}".format(accuracy_score(train_labels, train_predictions)))

In [16]:
import pickle

In [17]:
with open("models/cnn_svm_logreg.pkl", "wb") as file:
    pickle.dump(voting_model, file)

In [18]:
with open("models/cnn_svm_logreg.pkl", "rb") as file:
    model = pickle.load(file)

In [19]:
class_names = ["angular_leaf_spot", "bean_rust", "healthy"]
test_pred = model.predict(test_features)

In [20]:
report = classification_report(test_labels, test_pred, target_names=class_names)
print(report)

                   precision    recall  f1-score   support

angular_leaf_spot       0.93      0.86      0.89        43
        bean_rust       0.91      0.67      0.77        43
          healthy       0.75      1.00      0.86        42

         accuracy                           0.84       128
        macro avg       0.86      0.84      0.84       128
     weighted avg       0.86      0.84      0.84       128

